<a href="https://colab.research.google.com/github/DarkEol/AutoML-Ontology/blob/main/AutoSki_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

script for extraction of meta-features from datasets along with information about selected algorithm 

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 35.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 KB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 KB 6.0 MB/s eta 0:00:00
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641946 sha256=e9017d9f0e54d9588aa1b542036fd661bce3ae2f12720370661de90c6fc2a863

In [ ]:
import autosklearn.classification
import sklearn.model_selection
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

empty_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#AutoML settings
#cls = autosklearn.classification.AutoSklearnClassifier(
#    ensemble_class=None,
#    time_left_for_this_task=600
#)

In [ ]:
if __name__ == '__main__':
  mfeatures = pd.DataFrame(columns=["Name", "NumberOfInstances", "NumberOfFeatures", "ManyInstances", "FewInstances", "ManyFeatures", "FewFeatures", "BinaryClass",	"DateClass", "MissingClassValues", "NominalClass", "NumericClass", "NoClass",	"UnaryClass", "EmptyNominalClass", "StringClass", "RelationalClass", "OnlyMulti-InstanceData", "BinaryAttributes", "DateAttributes", "EmptyNominalAttributes", "MissingValues", "NominalAttributes", "NumericAttributes", "UnaryAttibutes",	"RelationalAttributes",	"StringAttributes", "Algorithm", "Accuracy"])
  #datasets_nums = [6,42,50,554,1596,23512]
  #datasets_nums = [23512] #554,1596 не прошли
  datasets_nums = [1480, 1485, 1487, 1489, 1491, 1494, 1497]
  i=0

  for dset_num in datasets_nums:
    opml = fetch_openml(data_id=dset_num)
    #data = opml.data
    X = opml.data
    y = opml.target
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=1)

    name = opml.details['name']
    print("name:", name)
    print("X size: ", len(X))
    print("X train size", len(X_train))
    print("X test size: ", len(X_test))

    #target = opml.target
    shape = X.shape
    n_instances = shape[0]
    n_features = shape[1]
    n_target_values = len(y.unique())

    numeric_data = X.select_dtypes(include=[np.number])
    #print("num_data:",numeric_data.shape[1])
    not_numeric_data = X.select_dtypes(exclude=[np.number])
    #print("not_num_data:",not_numeric_data.shape[1])
    category_data = X.select_dtypes(include=['category'])
    #print("category_data:",category_data.shape[1])
    
    mfeatures.loc[len(mfeatures)] = empty_list #adding row 

    mfeatures.at[i,'Name'] = name
    mfeatures.at[i,'NumberOfInstances'] = n_instances
    mfeatures.at[i,'NumberOfFeatures'] = n_features
  
    if n_instances > 3200:
      mfeatures.at[i,'ManyInstances'] = 1
    else:
      mfeatures.at[i,'FewInstances'] = 1
  
    if n_features > 100:
      mfeatures.at[i,'ManyFeatures'] = 1
    else:
      mfeatures.at[i, 'FewFeatures'] = 1

    if n_target_values == 2:
      mfeatures.at[i, 'BinaryClass'] = 1
      print("Binary")
    else:
      print("MultiClass")

    if y.str.isnumeric().any():
      mfeatures.at[i, 'NumericClass'] = 1
    else:
      mfeatures.at[i, 'StringClass'] = 1

    if y.dtype=='category':
      mfeatures.at[i, 'NominalClass'] = 1

    if numeric_data.shape[1] > 0:
      mfeatures.at[i, 'NumericAttributes'] = 1
  
    if category_data.shape[1] > 0:
      mfeatures.at[i, 'NominalAttributes'] = 1

    for fname in opml.feature_names:
      if len(X[fname].unique())==2: # and not binary_attributes:
        mfeatures.at[i, 'BinaryAttributes'] = 1

      if len(X[fname].unique())==1:
        mfeatures.at[i, 'UnaryAttibutes'] = 1

      if X[fname].isna().sum() > 0:
        mfeatures.at[i, 'MissingValues'] = 1

      if not pd.api.types.is_numeric_dtype(X[fname]):
        mfeatures.at[i, 'StringAttributes'] = 1

    #AutoML settings
    cls = autosklearn.classification.AutoSklearnClassifier(ensemble_class=None, time_left_for_this_task=600)

    #running search for best model
    print("searching for model")
    cls.fit(X_train, y_train)
    print("model found")

    #info about found models
    #print("leaderboard")
    #print(cls.leaderboard())
    algo = cls.leaderboard().iat[0,2]
    score = cls.score(X_test, y_test)

    print("algo: ",algo)
    print("score: ",score)
    mfeatures.at[i, 'Algorithm'] = algo
    mfeatures.at[i, 'Accuracy'] = score
    i+=1

name: ilpd
X size:  583
X train size 466
X test size:  117
Binary
searching for model


/usr/local/lib/python3.9/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


[WARNING] [2023-03-23 15:39:39,601:Client-AutoML(1):8cfc602a-c98f-11ed-8176-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 91, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "8cfc602a-c98f-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.3116883116883117, time=2.6990723609924316, status=<StatusType.SUCCESS: 1>, starttime=1679585393.0562367, endtime=1679585395.7721357, additional_info={'duration': 2.578990936279297, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "8cfc602a-c98f-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.27922077922077926, time=1.6182105541229248, status=<StatusType.SUCCESS: 1>, starttime=1679585395.776542, endtime=1679585397.4096956, additional_info={'duration': 1.5289301872253418, 'num_run': 3, 'train_loss': 0.22115384615384615, 'configuration_origin': 'Initial design'})
RunKey(conf

/usr/local/lib/python3.9/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


[WARNING] [2023-03-23 15:49:49,146:Client-AutoML(1):f51cb829-c990-11ed-8176-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 6, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "f51cb829-c990-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.3042212518195051, time=11.521968603134155, status=<StatusType.SUCCESS: 1>, starttime=1679585997.3013802, endtime=1679586008.8458338, additional_info={'duration': 11.414528608322144, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "f51cb829-c990-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.11790393013100442, time=11.748456954956055, status=<StatusType.SUCCESS: 1>, starttime=1679586008.851414, endtime=1679586020.6237242, additional_info={'duration': 11.587385416030884, 'num_run': 3, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=3, instanc

...

[WARNING] [2023-03-23 16:20:04,685:Client-AutoML(1):32ab9fd0-c995-11ed-8176-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 3, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "32ab9fd0-c995-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.1749408983451537, time=6.1115334033966064, status=<StatusType.SUCCESS: 1>, starttime=1679587817.6741803, endtime=1679587823.8216314, additional_info={'duration': 5.3794941902160645, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "32ab9fd0-c995-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.15130023640661938, time=5.1503660678863525, status=<StatusType.SUCCESS: 1>, starttime=1679587823.8275235, endtime=1679587829.017345, additional_info={'duration': 4.395442962646484, 'num_run': 3, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=3, instance

/usr/local/lib/python3.9/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


[WARNING] [2023-03-23 16:30:06,657:Client-AutoML(1):99b5c1f3-c996-11ed-8176-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 6, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "99b5c1f3-c996-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.12903225806451613, time=2.202712297439575, status=<StatusType.SUCCESS: 1>, starttime=1679588420.0328312, endtime=1679588422.2746296, additional_info={'duration': 2.050163507461548, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "99b5c1f3-c996-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.12186379928315416, time=2.763421058654785, status=<StatusType.SUCCESS: 1>, starttime=1679588422.281748, endtime=1679588425.0742772, additional_info={'duration': 2.61333966255188, 'num_run': 3, 'train_loss': 0.07256637168141589, 'configuration_origin': 'Initial design'})
RunKey(config_i

/usr/local/lib/python3.9/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


[WARNING] [2023-03-23 16:40:08,116:Client-AutoML(1):ffc2accb-c997-11ed-8176-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 3, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "ffc2accb-c997-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.006939625260235949, time=6.074631214141846, status=<StatusType.SUCCESS: 1>, starttime=1679589020.5664315, endtime=1679589026.6676826, additional_info={'duration': 5.9489850997924805, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "ffc2accb-c997-11ed-8176-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.005551700208188737, time=2.524207353591919, status=<StatusType.SUCCESS: 1>, starttime=1679589026.6724303, endtime=1679589029.2238553, additional_info={'duration': 2.4260241985321045, 'num_run': 3, 'train_loss': 0.0006842285323297448, 'configuration_origin': 'Initial design'})
RunKey(

In [ ]:
print(type(cls.leaderboard()))
print(cls.leaderboard())
print(cls.leaderboard().columns)

algo = cls.leaderboard().iat[0,2]
score = cls.score(X_test, y_test)

print("algo: ",algo)
print("score: ",score)
print(mfeatures)

<class 'pandas.core.frame.DataFrame'>
          rank  ensemble_weight               type      cost  duration
model_id                                                              
3            1              1.0  gradient_boosting  0.005552  2.524207
Index(['rank', 'ensemble_weight', 'type', 'cost', 'duration'], dtype='object')
algo:  gradient_boosting
score:  0.9954212454212454
                        Name  NumberOfInstances  NumberOfFeatures  \
0                       ilpd                583                10   
1                    madelon               2600               500   
2            ozone-level-8hr               2534                72   
3                    phoneme               5404                 5   
4  one-hundred-plants-margin               1600                64   
5                qsar-biodeg               1055                41   
6      wall-robot-navigation               5456                24   

   ManyInstances  FewInstances  ManyFeatures  FewFeatures  Binary

In [ ]:
print(mfeatures[['Name','ManyInstances', 'FewInstances', 'ManyFeatures', 'FewFeatures', 'BinaryClass','NumericClass','StringClass','NominalClass','BinaryAttributes','MissingValues','NominalAttributes','NumericAttributes','StringAttributes','UnaryAttibutes','Algorithm','Accuracy']])
print(mfeatures.shape)
mfeatures.to_csv('features.csv')
mfeatures.to_excel('features.xlsx')

                        Name  ManyInstances  FewInstances  ManyFeatures  \
0                       ilpd              0             1             0   
1                    madelon              0             1             1   
2            ozone-level-8hr              0             1             0   
3                    phoneme              1             0             0   
4  one-hundred-plants-margin              0             1             0   
5                qsar-biodeg              0             1             0   
6      wall-robot-navigation              1             0             0   

   FewFeatures  BinaryClass  NumericClass  StringClass  NominalClass  \
0            1            1             1            0             1   
1            0            1             1            0             1   
2            1            1             1            0             1   
3            1            1             1            0             1   
4            1            0            